# Monte-Carlo-Simulation

© Thomas Robert Holy 2019
<br>
Version 0.1.4

## Grundlegende Einstellungen:

Zunächst müssen die notwendigen Pakete (auch Module) importiert werden, damit auf diese zugegriffen werden kann. 

In [ ]:
import scipy.stats as st # SciPy ist ein Python-basiertes Ökosystem für Open-Source-Software für Mathematik, Naturwissenschaften und Ingenieurwissenschaften
from scipy.stats import rankdata, norm  
from scipy import array, linalg, dot
import random # Dieses modul wird verwendet um Zufallszahlen zu ziehen
import numpy as np # Programmbibliothek die eine einfache Handhabung von Vektoren, Matrizen oder generell großen mehrdimensionalen Arrays ermöglicht
import math # Dieses Modul wird verwendet um Skalardaten zu berechnen, z. B. trigonometrische Berechnungen.
import operator # Programmbibliothek, welche die Ausgaben übersichtlicher gestaltet
import matplotlib.pyplot as plt # Programmbibliothek die es erlaubt mathematische Darstellungen aller Art anzufertigen
import matplotlib.patches as mpatches

Anschließend werden Einstellungen definiert, die die Formatierung der Ausgaben betreffen.
Hierfür wird das Modul `operator` genutzt.
Außerdem wird die Breite des im Folgenden genutzten DataFrames erhöht und die Größe der Grafiken modifiziert, welche später angezeigt werden sollen.

In [ ]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

SCREEN_WIDTH = 114
centered = operator.methodcaller('center', SCREEN_WIDTH)

plt.rcParams["figure.figsize"] = 15,15

## Monte-Carlo Simulation:

### Variablen spezifizieren

Im ersten Schritt werden die für die Simulation notwenigen Variablen definiert.

In [ ]:
#----------------------------------------------------
# Anzahl Simulationsdurchläufe
n = 10000

#----------------------------------------------------
# Neue Randverteilungen (Gleichverteilung)
rand_x = [10,20]
rand_y = [8,22]

#----------------------------------------------------
# Varianzen/Standardabweichungen und Korrelation(en)
var_x = 4 
var_y = 9

corr_list = [0]
std_list = [math.sqrt(var_x),math.sqrt(var_y)]

#----------------------------------------------------
# Erwartungswerte
mu = [2, 3]

### Funktionen definieren

Als nächstes werden Funktionen definiert, welche aus den gegeben Varianzen und ggf. Korrelationen eine Varianz-Kovarianz-Matrix berechnen, ein Array mit den Varianzen anlegen und die Cholesky-Zerlegung durchführen.

In [ ]:
#---------------------------------------------------------------------------------------------------------------
# Varianz-Kovarianz-Matrix erstellen

def var_covar_matrix_func(std,corr):
    global var_covar
    global var_list
    
    i = 0
    j = 0
    counter0 = 0
    counter1 = 0    
    len_std_list = len(std_list)
    array = [[0] * len_std_list] * len_std_list    
    val_list = []
    
    for i in range(0,len_std_list):
        for j in range(0,len_std_list):
            if i == j:
                val = std_list[i] * std_list[i]
                val_list.append(val)
            if i < j:
                val = (std_list[i] * std_list[j] * corr[counter0])
                counter0 += 1
                val_list.append(val)
            if i > j:
                val = (std_list[i] * std_list[j] * corr[counter1])
                counter1 += 1
                val_list.append(val)
    var_covar = np.array(val_list).reshape(len_std_list,len_std_list)   
    
#---------------------------------------------------------------------------------------------------------------
# Varianz-Array erstellen

def var_func():
    global var_list
    var_list = []    
    for i in range(0,len(std_list)):
        var = np.power(std_list[i],2)
        var_list.append(var)

#---------------------------------------------------------------------------------------------------------------
# Cholesky-Zerlegung durchführen

def cholesky_func():
    global cholesky
    cholesky = linalg.cholesky(var_covar, lower=True)

#---------------------------------------------------------------------------------------------------------------   
# Funktionen aufrufen

var_func()
var_covar_matrix_func(std_list,corr_list)
cholesky_func()

### Monte-Carlo-Funktion definieren

Nun wird eine Funktion definiert, über welche die Simulation durchgeführt wird. 
Dabei werden zunächst zwei gleichverteilte Pseudo-Zufallsvariablen generiert, welche anschließend in unabhängige standardnormalverteilte Zufallsvariablen transformiert werden.
Letztere werden unter Anwendung der Cholesky-Zerlegung in abhängige standardnormalverteilte Zufallsvariablen überführt.
Danach werden diese unter Anwendung der oben definierten Standardabweichung und des Erwartungswertes in Realisationen der Gauss-Copula überführt, bevor auf diese anschließend die neuen Randverteilungen angewendet werden.
Zum Schluss wird die Summe aus beiden Realisationen gebildet, welche den Portfolio-Wert ergibt.

In [ ]:
total_standard_norm_ab_list = []
total_realisation_cop_list = []
total_xy_list = []
total_summe_liste = []

def copula_sim(n, rand_x, rand_y, mu):
    global total_standard_norm_ab_list
    global total_realisation_cop_list
    global total_xy_list
    global total_summe_liste
    
    for i in range(0,n):        
        #---------------------------------------------------------------------------------------------------------------
        # Gleichverteilte Zufallsvariablen ziehen
        
        n = 2
        random_ZV_list = []        
        for i in range(0,n):
            x = random.random()
            random_ZV_list.append(x)
        
        #---------------------------------------------------------------------------------------------------------------
        # Transformation der gleichverteilten Zufallsvariablen in unabhängige standardnormalverteilte Zufallsvariablen

        standard_norm_list = norm.ppf(random_ZV_list)        
        
        #---------------------------------------------------------------------------------------------------------------
        # Transformation in standardnormalvertzeilte abhängie ZV
        
        standard_norm_ab_list = []
        counter_cholesky_0 = 0
        counter_cholesky_1 = 0
        counter_mu = 0
        counter_standard_norm_list = 0

        for i in range(0,2):
            a = cholesky[counter_cholesky_0][counter_cholesky_1] * standard_norm_list[counter_standard_norm_list]
            counter_cholesky_1 += 1
            counter_standard_norm_list += 1

            b = cholesky[counter_cholesky_0][counter_cholesky_1] * standard_norm_list[counter_standard_norm_list]
            counter_cholesky_0 += 1  
            counter_cholesky_1 = 0
            counter_standard_norm_list = 0  

            r = a  + b + mu[counter_mu]
            standard_norm_ab_list.append(r)
            counter_mu += 1

        #---------------------------------------------------------------------------------------------------------------
        # Transformation Realisationen der Gauss-Copula

        counter = 0
        realisation_cop_list = []        
        for i in range(0,2):
            r_cop = (standard_norm_ab_list[counter] - mu[counter]) / math.sqrt(var_list[counter])   
            p_value = st.norm.cdf(r_cop)
            realisation_cop_list.append(p_value)
            counter += 1
            
        #---------------------------------------------------------------------------------------------------------------
        # Gemeinsame Verteilung: Übertragung der neuen Ränder auf die ermittelte Abhängigkeitsstruktur          

        x = (rand_x[0]-rand_x[1]) * realisation_cop_list[0] + rand_x[1]
        y = (rand_y[0]-rand_y[1]) * realisation_cop_list[1] + rand_y[1]
        summe = x + y        
        xy = (x,y)
        
        total_standard_norm_ab_list.append(standard_norm_ab_list)
        total_realisation_cop_list.append(realisation_cop_list)
        total_xy_list.append(xy)
        total_summe_liste.append(summe)
        
copula_sim(n, rand_x, rand_y, mu)

# Graphische Ausgaben

## Realisationen abhängiger bivariat normalverteilter Zufallsvariablen

In [ ]:
#----------------------------------------------------------
# Realisationen abhängiger bivariat normalverteilter Zufallsvariablen

counter_1 = 0
counter_2 = 0
x_liste = []
y_liste = []

for i in range(0,len(total_standard_norm_ab_list)):
    x = total_standard_norm_ab_list[counter_1][0]
    y = total_standard_norm_ab_list[counter_1][1]
    x_liste.append(x)
    y_liste.append(y)
    counter_1 += 1
    counter_2 += 1

plt.scatter(x_liste, y_liste)
plt.grid()
plt.xlabel('Realisation X')
plt.ylabel('Realisation Y')
plt.title('Realisationen abhängiger bivariat normalverteilter Zufallsvariablen')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.show()

## Realisationen Gauss-Copula

In [ ]:
#----------------------------------------------------------
# Realisationen Gauss-Copula

counter_1 = 0
counter_2 = 0
x_liste = []
y_liste = []

for i in range(0,len(total_realisation_cop_list)):
    x = total_realisation_cop_list[counter_1][0]
    y = total_realisation_cop_list[counter_1][1]
    x_liste.append(x)
    y_liste.append(y)
    counter_1 += 1
    counter_2 += 1

plt.scatter(x_liste, y_liste)
plt.grid()
plt.xlabel('Realisation X')
plt.ylabel('Realisation Y')
plt.title('Realisationen Gauss-Copula')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.show()


## Realisationen gleichverteilte X, Y mit Gauss-Copula

In [ ]:
#----------------------------------------------------------
# Realisationen gleichverteilte X, Y mit Gauss-Copula

counter_1 = 0
counter_2 = 0
x_liste = []
y_liste = []

for i in range(0,len(total_xy_list)):
    x = total_xy_list[counter_1][0]
    y = total_xy_list[counter_1][1]
    x_liste.append(x)
    y_liste.append(y)
    counter_1 += 1
    counter_2 += 1

plt.scatter(x_liste, y_liste)
plt.grid()
plt.xlabel('Realisation X')
left, right = plt.xlim()
plt.xlim((left, right))   
plt.xlim(left, right)     
left, right = plt.ylim()
plt.ylim((left, right)) 
plt.ylim(left, right)     
plt.ylabel('Realisation Y')
plt.title('Realisationen gleichverteilte X, Y mit Gauss-Copula')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.show()

## Histogramm X+Y, Realisationen gleichverteilte X, Y mit Gauss-Copula

In [ ]:
#----------------------------------------------------------
# Histogramm X+Y, Realisationen gleichverteilte X, Y mit Gauss-Copula

values_PF = total_summe_liste
bins = n

H, X1 = np.histogram(values_PF, bins, density=True)
dx = X1[1] - X1[0]
F1 = np.cumsum(H)*dx
plt.plot(X1[1:], F1)

plt.grid()
plt.xlabel('PF-Realisation')
plt.ylabel('Wahrscheinlichkeit')
left, right = plt.xlim()
plt.xlim((left, right))
plt.xlim(left, right)
blue_patch = mpatches.Patch(color='blue', label='Monte-Carlo Simulation')
plt.legend(handles=[blue_patch], loc='upper left')
plt.title('Histogram X+Y, Realisationen gleichverteilte X,Y mit Gauss-Copula')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.show()

## Verteilung X+Y, Realisationen gleichverteilte X, Y mit Gauss-Copula

In [ ]:
#----------------------------------------------------------
# Verteilung X+Y, Realisationen gleichverteilte X, Y mit Gauss-Copula

bins = 71

data = total_summe_liste
plt.hist(data, bins)
plt.grid()
plt.title('Verteilung X+Y, Realisationen gleichverteilte X,Y mit Gauss-Copula')
plt.xlabel('PF-Realisation')
plt.ylabel('Anzahl')
plt.show()

## Risikomaäße schätzen

### Value at Risk

Um den Value at Risk zu bestimmen, werden die Portfolio-Realisationen zunächst der Größe nach sortiert, wobei dieser Schritt gleichzeitig der Ausgangspunkt für die Berechnung des Conditional Value at Risk ist.
Anschließend wird das Alpha-Quantil der Verlustfunktion bestimmt, indem der Parameter "alpha" mit der Länge der Liste "RM_list" multipliziert wird. Der so ermittelte Wert gibt die Position in der Liste "RM_list" an, an welcher sich der Value at Risk zum Konfidenzniveau "alpha" befindet.

In [ ]:
RM_list = sorted(values_PF)

def VaR(alpha):
    item = int((alpha * len(RM_list)))
    VaR = RM_list[item]
    print('#------------------------------------------------------------------------------------------------------------------#')
    print('|' + centered('Der VaR beträgt: ' + str(VaR) + '.') + '| ')

### Conditional Value at Risk

Der Conditional Value at Risk wird grundsätzlich wie der Value at Risk bestimmt, jedoch wird hier der Mittelwert über alle Realisationen bis zum Alpha-Quantil bebildet. 
Daher wird hier nach der Positionsbestimmung eine Liste mit Realisationen generiert, welche den Bereich von der Kleinsten bis zum Alpha-Quantil abdeckt. Diese Liste wird anschließend durch die Anzahl ihrer Elemente geteilt.

In [ ]:
def CVaR(alpha):
    item = int((alpha * len(RM_list)))
    CVaR_list = RM_list[0:item]     
    CVaR = np.sum(CVaR_list) / len(CVaR_list)
    print('#------------------------------------------------------------------------------------------------------------------#')
    print('|' + centered('Der CVaR beträgt: ' + str(CVaR) + '.') + '| ')

### Power-Spektrales Risikomaß

Für das Power-Spektrale Risikomaß ergibt sich der Erwartungswert aus dem Mittelwert der "RM_list" (der Mittelwert der Portfolio-Realisationen) und das Risiko ergibt sich aus dem Matrixprodukt der transponierten "RM_list" mit der "subj_ws_list", welche subjektive Wahrscheinlichkeiten beinhaltet.
Die Elemente letzterer Liste werden berechnet, indem jede Realisation (hier 1 bis 10.000) durch die Gesamtanzahl der Realisationen (hier: 10.000) geteilt und dann mit "gamma" potenziert wird (daher heißt es Power-Sepktrales Risikomaß).
Dabei ist bei jeder Berechnung der jeweils vorher errechnete Wert zu subtrahieren.

In [ ]:
def power(gamma):
    EW = np.mean(RM_list)
    print('#------------------------------------------------------------------------------------------------------------------#')
    print('|' + centered('Power-Spektrales Risikomaß bei der historischen Simulation:') + '| ')
    print('#------------------------------------------------------------------------------------------------------------------#')
    print('|' + centered('Der Erwartungswert beträgt: ' + str(EW) + '.') + '| ')

    subj_ws_list = []
    counter1 = len(RM_list)
    counter2 = len(RM_list)-1
    for i in RM_list:
        subj_ws = (np.power((counter1 / len(RM_list)), gamma)) - (np.power((counter2 / len(RM_list)), gamma)) 
        counter1 -= 1
        counter2 -= 1
        subj_ws_list.append(subj_ws)
    subj_ws_list = subj_ws_list[::-1]
    risk = np.matmul(np.transpose(RM_list), subj_ws_list)
    print('|' + centered('Das Risiko beträgt: ' + str(risk) + '.') + '| ')
    print('#------------------------------------------------------------------------------------------------------------------#')

### Parameterfestlegung und Aufruf der Funktionen

In [ ]:
alpha = 0.05
VaR(alpha)

alpha = 0.05
CVaR(alpha)

gamma = 0.5
power(gamma)

## Inkonsistenz Monte-Carlo Simulation

Eins der Probleme der Monte-Carlo Simulation ist deren Inskonstenz sofern nur wenige Durchführungen in einem Simulationskauf vorgenommen werden.
Dies soll die folgende Grafik veranschaulichen.

In [ ]:
total_summe_liste = []

#-------------------------------------------
# Variablen für den Simulationslauf

rand_x = [10,20]
rand_y = [8,22]
var_x = 4 
var_y = 9
corr_list = [0]
std_list = [math.sqrt(var_x),math.sqrt(var_y)]
mu = [2, 3]

#-------------------------------------------
# Funktionsaufruf

var_func()
var_covar_matrix_func(std_list,corr_list)
cholesky_func()

#-------------------------------------------------------------------------
# Wiederholungen der Simulationen

runs_sim = 100 # Legt die Anzahl der Durchläufe einer Simulation fest
runs_func = 5 # Legt fest, wie viele Simulationen durchgeführt werden

for i in range(0,runs_func):
    copula_sim(runs_sim, rand_x, rand_y, mu)

#-------------------------------------------
counter0 = 0
counter1 = runs_sim
array = []
for i in range(0,runs_func):
    x = total_summe_liste[counter0:counter1]
    array.append(x)
    counter0 += runs_sim
    counter1 += runs_sim

In [ ]:
for items in array:
    values_PF = items
    bins = runs_sim

    H, X1 = np.histogram(values_PF, bins, density=True)
    dx = X1[1] - X1[0]
    F1 = np.cumsum(H)*dx
    plt.plot(X1[1:], F1)

plt.grid()
plt.xlabel('PF-Realisation')
plt.ylabel('Wahrscheinlichkeit')
left, right = plt.xlim()
plt.xlim((left, right))
plt.xlim(left, right)
black_patch = mpatches.Patch(color='black', label='Monte-Carlo Simulationen')
plt.legend(handles=[black_patch], loc='upper left')
plt.title('Histogram X+Y, Realisationen gleichverteilte X,Y mit Gauss-Copula')
plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.show()